<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S101lemHYstanzaOCRv02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Armenian lemmatization with Stanza
https://github.com/iued-uni-heidelberg/corpustools/blob/main/S101lemHYstanzaOCRv01.ipynb


In [ ]:
import os, sys, re


## downloading evaluation sets
- 420 words: test with about 420 words of Armenian text
- Armenian "Brown-type" corpus b

In [ ]:
### optional
!wget https://heibox.uni-heidelberg.de/f/ce6096da570f47b99500/?dl=1
### optional
!mv index.html?dl=1 evaluation-set-v01.txt

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/a847a12bffd4491f9070/?dl=1
!mv index.html?dl=1 TED2020-dehy-hy-aa

In [ ]:
### downloading Armenian Wikipedia
!wget https://heibox.uni-heidelberg.de/f/d1f866a61bd545318213/?dl=1
!mv index.html?dl=1 hywiki-20221101-pages-articles.txt.gz
!gunzip hywiki-20221101-pages-articles.txt.gz

--2023-03-20 08:45:40--  https://heibox.uni-heidelberg.de/f/d1f866a61bd545318213/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/12517929-9d0e-47db-a15e-844e700ac3db/hywiki-20221101-pages-articles.txt.gz [following]
--2023-03-20 08:45:40--  https://heibox.uni-heidelberg.de/seafhttp/files/12517929-9d0e-47db-a15e-844e700ac3db/hywiki-20221101-pages-articles.txt.gz
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 193021637 (184M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 184.08M  14.9MB/s    in 12s     

2023-03-20 08:45:52 (15.7 MB/s) - ‘index.html?dl=1’ saved [193021637/193021637]



In [ ]:
!wc hywiki-20221101-pages-articles.txt

  2446411  56341171 803098410 hywiki-20221101-pages-articles.txt


## Installing stanza

In [ ]:
!pip install spacy-stanza

In [ ]:
import stanza
import spacy_stanza


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


### testing English stanza (optional)

In [ ]:
# optional
# Download the stanza model if necessary
stanza.download("en")

# Initialize the pipeline
nlp = spacy_stanza.load_pipeline("en")

doc = nlp("Barack Obama was born in Hawaii. He was elected president in 2008.")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type_)
print(doc.ents)

### downloading and testing Armenian stanza

In [ ]:
stanza.download("hy")


INFO:stanza:Downloading default packages for language: hy (Armenian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp_hy = spacy_stanza.load_pipeline("hy")

In [ ]:
### optional
doc = nlp_hy("ՄԱՐԴՈՒ ԻՐԱՎՈՒՆՔՆԵՐԻ ՀԱՄԸՆԴՀԱՆՈՒՐ ՀՌՉԱԿԱԳԻՐ. ՆԵՐԱԾԱԿԱՆ. Քանզի մարդկային ընտանիքի բոլոր անդամներին ներհատուկ արժանապատվությունըև հավասար ու անօտարելի իրավունքները աշխարհի ազատության, արդարության ու խաղաղության հիմքն են.")

In [ ]:
### optional
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type_)


### full analysis of the file (optional)
- includes dependency parsing

In [ ]:
### optional
with open('/content/TED2020-dehy-hy-aa', 'r', encoding='utf-8') as infile, open('/content/TED2020-dehy-hy-aa-ANALYSIS-full-v01.txt', 'w') as outfile:
    # read sample.txt an and write its content into sample2.txt
    outfile.write("{token.text}\t{token.lemma_}\t{token.pos_}\t{token.dep_}\t{parentLem}\t{LAncestors}\n")
    for line in infile:
        line = line.strip()
        doc = nlp_hy(line)
        # outfile.write(line + '\n')
        for token in doc:
            LAncestors = list(token.ancestors)
            print(str(LAncestors))
            try:
                SLAncestors = str(list(token.ancestors))
                parent = LAncestors[0]
                parentLem = parent.lemma_
            except:
                parentLem = "NONE"
            outfile.write(f"{token.text}\t{token.lemma_}\t{token.pos_}\t{token.dep_}\t{parentLem}\t{SLAncestors}\n")
 

 

In [ ]:
### optional : check output 
!head -n 50 TED2020-dehy-hy-aa-ANALYSIS-full-v01.txt

head: cannot open 'TED2020-dehy-hy-aa-ANALYSIS-full-v01.txt' for reading: No such file or directory


### function for lemmatization

In [ ]:
def parseFile(iFileName, oFileName, nlp_model = nlp_hy):
    with open(iFileName, 'r', encoding='utf-8') as infile, open(oFileName, 'w') as outfile:
        # read sample.txt an and write its content into sample2.txt
        outfile.write("{token.text}\t{token.pos_}\t{token.lemma_}\n")
        c = 0
        for line in infile:
            c+=1
            if c%10 == 0: print(str(c))
            line = line.strip()
            doc = nlp_model(line)
            # outfile.write(line + '\n')
            for token in doc:
                LAncestors = list(token.ancestors)
                # print(str(LAncestors))
                try:
                    SLAncestors = str(list(token.ancestors))
                    parent = LAncestors[0]
                    parentLem = parent.lemma_
                except:
                    parentLem = "NONE"
                outfile.write(f"{token.text}\t{token.pos_}\t{token.lemma_}\n")
        outfile.flush()
    return


### command to lemmatize the file

In [ ]:
# 1000 lines, runs in 2 minutes...
parseFile('/content/TED2020-dehy-hy-aa', '/content/TED2020-dehy-hy-aa--lemmatization-v01.txt', nlp_hy)

In [ ]:
# takes a lot of time, do not run it, just use as a template, split the file first and run several processes...
parseFile('hywiki-20221101-pages-articles.txt', 'hywiki-20221101-pages-articles.vert', nlp_hy)

## Checking OCR errors
### wikipedia lemmatized --> frequency dictionary 

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/5b3213f991f84ca496ba/?dl=1
!mv index.html?dl=1 hywiki-20221101-pages-articles-v03.vert

In [ ]:
!wc hywiki-20221101-pages-articles-v03.vert

 2735467  8206467 75483279 hywiki-20221101-pages-articles-v03.vert


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/350790e66ca24efdab1a/?dl=1
!mv index.html?dl=1 hy-texts-vert.tgz 
!tar xvzf hy-texts-vert.tgz

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/d601ceb0af5a4671a8e7/?dl=1
!mv index.html?dl=1 Parfum_Arm_ABBY.txt

In [ ]:
!wc Parfum_Arm_ABBY.txt

In [ ]:
parseFile('Parfum_Arm_ABBY.txt', 'Parfum_Arm_ABBY.vert.txt', nlp_hy)

In [ ]:
!wc Parfum_Arm_ABBY.vert.txt

  83890  251594 2098541 Parfum_Arm_ABBY.vert.txt


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/743a1a57a37c42d8b585/?dl=1
!mv index.html?dl=1 Parfum_Armenian_uncorrected.txt


In [ ]:
!wc Parfum_Armenian_uncorrected.txt

 13592  72207 854251 Parfum_Armenian_uncorrected.txt


In [ ]:
# dealing with Armenian OCR output with line breaks (is it correct?)

FName = 'Parfum_Armenian_uncorrected.txt'
FNameOut = 'Parfum_Armenian.txt'

FIn = open(FName, 'r')
FOut = open(FNameOut, 'w')

for SLine in FIn:
    SLine = SLine.strip()
    if SLine == '': 
        FOut.write('\n\n')
        continue
    if SLine[-1] == '-':
        SLine2write = SLine[:-1]
        FOut.write(SLine2write)
        continue

    FOut.write(SLine + ' ')
FOut.flush()



In [ ]:
!wc Parfum_Armenian.txt

  6126  69006 849250 Parfum_Armenian.txt


In [ ]:
# runs for approximately 10 minutes, 6126 lines, 83828 words
parseFile('Parfum_Armenian.txt', 'Parfum_Armenian.vert.txt', nlp_hy)

10
20
30
40
50
60


/usr/local/lib/python3.9/dist-packages/spacy/language.py:999: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  doc = self._ensure_doc(text)
/usr/local/lib/python3.9/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ["'", 'Շարցնում', 'են', "'", 'ինչ', '՞', 'է', 'եղել', 'նրա', 'հետ', ':']
Entities: []
  doc = self._ensure_doc(text)
/usr/local/lib/python3.9/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['-', 'ինչ', '՛', 'էնա', 'անում', 'դանակով', ':']
Entities: []
  doc = self._ensure_doc(text)


70
80
90
100
110


/usr/local/lib/python3.9/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['-', 'Նրա', 'համար', 'վնասակար', 'չի', 'լինի', ',', '-', 'շշպռեց', 'Ժաննան', ',', '-', 'իսկ', 'ինձ', 'համար', 'կլինի', ':', 'Ես', 'նիհարել', 'եմ', 'տասը', 'ֆունտ', ',', 'չնայած', 'կերել', 'եմ', 'երեք', 'հոգու', 'փոխարեն', ':', 'Իսկ', 'հանուն', 'ինչի', ':', 'Հանուն', 'շաբաթական', 'երեք', '<UNK>րանկի', '՛']
Entities: [('Ժաննան', 'PERSON', 38, 44), ('տասը', 'CARDINAL', 82, 86), ('երեք', 'CARDINAL', 110, 114), ('երեք', 'CARDINAL', 164, 168)]
  doc = self._ensure_doc(text)


120


...

2060
2070
2080
2090
2100
2110
2120
2130
2140


/usr/local/lib/python3.9/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['ակելով', 'թաշկինակը', 'սեղմել', 'քթին', ',', 'կարծես', 'ուզում', 'էր', 'ով', 'պաշտպանվել', 'իր', 'հոգու', 'վրաանո', '՛', 'հարձակումից', ':']
Entities: []
  doc = self._ensure_doc(text)


2150


/usr/local/lib/python3.9/dist-packages/spacy/language.py:999: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['-', 'Դուք', 'չեք', '՛', 'ցանկանում', 'փորձանմուշ', 'վերցնել', ',', '-', '-', 'կրկին', 'կարկաչող', 'ճայնով', 'ասաց', 'Գրենույը', ',', '-', 'միթե', '՛', 'չեք', 'ուզում', ',', 'վարպետ', ':', 'Միթե', '՛', 'չեք', 'փորձի', ':']
Entities: [('Գրենույը', 'PERSON', 71, 79)]
  doc = self._ensure_doc(text)


2160
2170
2180
2190
2200
2210
2220


In [ ]:
!wc Parfum_Armenian.vert.txt

  83828  251460 2055080 Parfum_Armenian.vert.txt


## Downloading full Armenian corpus

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/c977e87cf2b244e6801b/?dl=1
!mv index.html?dl=1 KorpusARM.tgz

In [ ]:
!tar xvzf KorpusARM.tgz

In [ ]:
!mkdir KorpusARM1

In [ ]:
mkdir KorpusARM1/stage01

In [ ]:
# concatenating files
!cat korpusARM/hyFiktion/* >KorpusARM1/stage01/hyFiktion.txt
!cat korpusARM/hyNatur/* >KorpusARM1/stage01/hyNatur.txt
!cat korpusARM/hyRecht/* >KorpusARM1/stage01/hyRecht.txt

In [ ]:
!mkdir KorpusARM1/stage02

In [ ]:
# function for Armenian line breaks:

def correctLineBreaksHY(FName, FNameOut):
    FIn = open(FName, 'r')
    FOut = open(FNameOut, 'w')
    countHyphens = 0
    for SLine in FIn:
        SLine = SLine.strip()
        if SLine == '': 
            FOut.write('\n\n')
            continue
        if SLine[-1] == '-':
            SLine2write = SLine[:-1]
            FOut.write(SLine2write)
            countHyphens +=1
            continue
        FOut.write(SLine + ' ')
    FOut.flush()
    print(str(countHyphens) + ' hyphens corrected')
    return



In [ ]:
correctLineBreaksHY('KorpusARM1/stage01/hyFiktion.txt', 'KorpusARM1/stage02/hyFiktion.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyNatur.txt', 'KorpusARM1/stage02/hyNatur.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyRecht.txt', 'KorpusARM1/stage02/hyRecht.txt')

In [ ]:
!wc KorpusARM1/stage02/hyFiktion.txt
!wc KorpusARM1/stage02/hyNatur.txt
!wc KorpusARM1/stage02/hyRecht.txt

In [ ]:
!mkdir KorpusARM1/stage03

In [ ]:
# runs for about 47 minutes; alternatively, excecute one of the following cells to download parsed archived corpus
parseFile('KorpusARM1/stage02/hyFiktion.txt', 'KorpusARM1/stage03/hyFiktion.vert.txt', nlp_hy)
parseFile('KorpusARM1/stage02/hyNatur.txt', 'KorpusARM1/stage03/hyNatur.vert.txt', nlp_hy)
parseFile('KorpusARM1/stage02/hyRecht.txt', 'KorpusARM1/stage03/hyRecht.vert.txt', nlp_hy)

In [ ]:
!tar cvzf KorpusARM1_stage03.tgz KorpusARM1/stage03/

KorpusARM1/stage03/
KorpusARM1/stage03/hyNatur.vert.txt
KorpusARM1/stage03/hyRecht.vert.txt
KorpusARM1/stage03/hyFiktion.vert.txt


In [ ]:
# alternative: download Lemmatized corpus
!wget https://heibox.uni-heidelberg.de/f/095d2385bad74f2b8ffd/?dl=1
!mv index.html?dl=1 KorpusARM1_stage03.tgz
!tar xvzf KorpusARM1_stage03.tgz

In [ ]:
!wc /content/KorpusARM1/stage03/hyFiktion.vert.txt
!wc /content/KorpusARM1/stage03/hyNatur.vert.txt
!wc /content/KorpusARM1/stage03/hyRecht.vert.txt



 115616  346846 2690663 /content/KorpusARM1/stage03/hyFiktion.vert.txt
  83271  249801 2110874 /content/KorpusARM1/stage03/hyNatur.vert.txt
 102399  307197 2970463 /content/KorpusARM1/stage03/hyRecht.vert.txt


In [ ]:
!cat /content/KorpusARM1/stage03/* >/content/KorpusARM1/stage03all.vert.txt

In [ ]:
!wc /content/KorpusARM1/stage03all.vert.txt

 301286  903844 7772000 /content/KorpusARM1/stage03all.vert.txt


## Creating dictionaries from parsed files

In [ ]:
DWiki = {}
with open("hywiki-20221101-pages-articles-v03.vert", 'r') as f:
    for line in f:
        line = line.rstrip()
        try:
            DWiki[line] +=1
        except:
            DWiki[line] = 1


In [ ]:
DText = {}
with open("Parfum_Armenian.vert.txt", 'r') as f:
    for line in f:
        line = line.rstrip()
        try:
            DText[line] +=1
        except:
            DText[line] = 1


## Functions for comparing texts with Wiki corpus

In [ ]:
def lines2dict(SFIn):
    DText = {}
    with open(SFIn, 'r') as f:
        for line in f:
            line = line.rstrip()
            try:
                DText[line] +=1
            except:
                DText[line] = 1
    print(len(DText))
    return DText

In [ ]:
def compareDicts(DText, DWiki, lenText, lenWiki):
    DFreqDiff = {} # dictionary of frequency differences
    # lenWiki = 2735468
    # lenText = 83829
    c = 0
    for key, val in sorted(DText.items(), key=lambda item: item[1], reverse=True):
        c+=1
        valText = val + 1
        relText = valText / lenText
        try:
            valWiki = DWiki[key] + 1
        except:
            valWiki = 1
        relWiki = valWiki / lenWiki

        diffValue = relText / relWiki
        DFreqDiff[key] = diffValue
    
    print(len(DFreqDiff))
    return DFreqDiff

In [ ]:
def printCompareDict(DFreqDiff, DText, DWiki, SFOut):
    fOut = open(SFOut, 'w')
    countEntries = 0
    for key, val in sorted(DFreqDiff.items(), key=lambda item: item[1], reverse=True):
        countEntries += 1
        try:
            frqText = DText[key] + 1
        except:
            frqText = 1

        try:
            frqWiki = DWiki[key] + 1
        except:
            frqWiki = 1
        fOut.write(f'{key}\t{val}\t{frqText}\t{frqWiki}\n')
    fOut.flush()
    print(countEntries)
    return


In [ ]:
DWikiX = lines2dict('/content/hywiki-20221101-pages-articles-v03.vert')

227720


In [ ]:
DhyFiktion = lines2dict('/content/KorpusARM1/stage03/hyFiktion.vert.txt')
DhyNatur = lines2dict('/content/KorpusARM1/stage03/hyNatur.vert.txt')
DhyRecht = lines2dict('/content/KorpusARM1/stage03/hyRecht.vert.txt')
DhyAll = lines2dict('/content/KorpusARM1/stage03all.vert.txt')

20969
21806
11064
43675


In [ ]:
DFreqDiffWikiFiktion = compareDicts(DhyFiktion, DWikiX, 115616, 2735468)
DFreqDiffWikiNatur = compareDicts(DhyNatur, DWikiX, 83271, 2735468)
DFreqDiffWikiRecht = compareDicts(DhyRecht, DWikiX, 102399, 2735468)
DFreqDiffWikiAll = compareDicts(DhyAll, DWikiX, 301286, 2735468)

20969
21806
11064
43675


In [ ]:
!mkdir KorpusARM1/stage04

In [ ]:
printCompareDict(DFreqDiffWikiFiktion, DhyFiktion, DWikiX, '/content/KorpusARM1/stage04/hyFiktion.tsv.txt')
printCompareDict(DFreqDiffWikiNatur, DhyNatur, DWikiX, '/content/KorpusARM1/stage04/hyNatur.tsv.txt')
printCompareDict(DFreqDiffWikiRecht, DhyRecht, DWikiX, '/content/KorpusARM1/stage04/hyRecht.tsv.txt')
printCompareDict(DFreqDiffWikiAll, DhyAll, DWikiX, '/content/KorpusARM1/stage04all.tsv.txt')

In [ ]:
# Dowloading the file with corrections
!wget https://heibox.uni-heidelberg.de/f/14706c04a4024b2f937d/?dl=1
!mv index.html?dl=1 Pilot-Corrections-all.tsv
!wc Pilot-Corrections-all.tsv


In [ ]:
def readCorrections(colNumberOri, colNumberCorrect, SFIn, SFOut = None):
    LTWrongCorrect = []
    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.strip()
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\n')    
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect

In [ ]:
def readCorrectionsFrq(colNumberOri, colNumberCorrect, colNumberFrq, SFIn, SFOut = None):
    LTWrongCorrect = []
    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.strip()
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            SFrq = LLine[colNumberFrq]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]', f'{SFrq}')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect, SFrq in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\t{SFrq}\n')    
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect

In [ ]:
LTWrongCorrectWord, DWrongCorrectWord = readCorrections(1, 4, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-WordForm.tsv')
LTWrongCorrectLemma, DWrongCorrectLemma = readCorrections(3, 6, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-Lemma.tsv')


In [ ]:
print(LTWrongCorrectWord)
print(LTWrongCorrectLemma)

[('[մերճակա]', '[մերձակա]'), ('[առջն]', '[առջև]'), ('[թեթնություն]', '[թեթևություն]'), ('[Եթենա]', '[եթե նա]'), ('[ննա]', '[նա]'), ('[ճեռքերն]', '[ձեռքից]'), ('[ճայն]', '[ձայն]'), ('[ճեռքով]', '[ձեռքով]'), ('[այլնս]', '[այլևս]'), ('[ետնի]', '[ետևի]'), ('[կեղնները]', '[կեղևները]'), ('[ճկան]', '[ձկան]'), ('[ննա]', '[նա]'), ('[բանաձնի]', '[բանաձևի]'), ('[ճեր]', '[ձեր]'), ('[առնտրական]', '[առևտրական]'), ('[արնելյան]', '[արևելյան]'), ('[կուղնորվի]', '[կուղևորվի]'), ('[նս]', '[ևս]'), ('[հետնեց]', '[հետևել]'), ('[նուխիսկ]', '[նույնիսկ]'), ('[երնույթ]', '[երևույթ]'), ('[քրտնքով]', '[քրտինքով]'), ('[արվարճանում]', '[արվարձանում]'), ('[անճամբ]', '[անձամբ]'), ('[ճայնը]', '[ձայնը]'), ('[ճգվում]', '[ձգվում]'), ('[ճիու]', '[ձիու]'), ('[դարճնում]', '[դարձնում]'), ('[ուղնորվում]', '[ուղևորվում ]'), ('[իջնանատան]', '[իջևանատան]'), ('[ճգտում]', '[ձգտում]'), ('[դրսնորում]', '[դրսևորում]'), ('[արվարճանի]', '[արվարձանի]'), ('[ետնում]', '[ետևում]'), ('[ճնավորված]', '[ձևավորված]'), ('[Հետնաբար]', '[հետևաբար]

In [ ]:
LTWrongCorrectWordF, DWrongCorrectWordF = readCorrectionsFrq(1, 4, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-WordForm.tsv')
LTWrongCorrectLemmaF, DWrongCorrectLemmaF = readCorrectionsFrq(3, 6, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-Lemma.tsv')

ինճ	ինձ	ինչ
առջնից	առջև	առջևից
ինճ	ինչ	ինձ
172
ինճ	ինչ	ես
գիտել	գիտենալ	իմանալ
206


In [ ]:
print(LTWrongCorrectWordF)
print(LTWrongCorrectLemmaF)

[('[մերճակա]', '[մերձակա]', '4'), ('[առջն]', '[առջև]', '4'), ('[թեթնություն]', '[թեթևություն]', '4'), ('[Եթենա]', '[եթե նա]', '4'), ('[ննա]', '[նա]', '4'), ('[ճեռքերն]', '[ձեռքից]', '4'), ('[ճայն]', '[ձայն]', '4'), ('[ճեռքով]', '[ձեռքով]', '4'), ('[այլնս]', '[այլևս]', '4'), ('[ետնի]', '[ետևի]', '4'), ('[կեղնները]', '[կեղևները]', '4'), ('[ճկան]', '[ձկան]', '4'), ('[ննա]', '[նա]', '4'), ('[բանաձնի]', '[բանաձևի]', '4'), ('[ճեր]', '[ձեր]', '4'), ('[առնտրական]', '[առևտրական]', '4'), ('[արնելյան]', '[արևելյան]', '4'), ('[կուղնորվի]', '[կուղևորվի]', '4'), ('[նս]', '[ևս]', '4'), ('[հետնեց]', '[հետևել]', '4'), ('[նուխիսկ]', '[նույնիսկ]', '4'), ('[երնույթ]', '[երևույթ]', '4'), ('[քրտնքով]', '[քրտինքով]', '4'), ('[արվարճանում]', '[արվարձանում]', '4'), ('[անճամբ]', '[անձամբ]', '4'), ('[ճայնը]', '[ձայնը]', '4'), ('[ճգվում]', '[ձգվում]', '4'), ('[ճիու]', '[ձիու]', '4'), ('[դարճնում]', '[դարձնում]', '4'), ('[ուղնորվում]', '[ուղևորվում ]', '4'), ('[իջնանատան]', '[իջևանատան]', '4'), ('[ճգտում]', '[ձգտո

In [ ]:
printCompareDict(DFreqDiffWikiFiktion, DhyFiktion, DWikiX, '/content/KorpusARM1/stage04/hyFiktion.tsv.txt')
printCompareDict(DFreqDiffWikiNatur, DhyNatur, DWikiX, '/content/KorpusARM1/stage04/hyNatur.tsv.txt')
printCompareDict(DFreqDiffWikiRecht, DhyRecht, DWikiX, '/content/KorpusARM1/stage04/hyRecht.tsv.txt')
printCompareDict(DFreqDiffWikiAll, DhyAll, DWikiX, '/content/KorpusARM1/stage04all.tsv.txt')

20969
21806
11064
43675


In [ ]:
!mkdir /content/KorpusARM1/stage05

In [ ]:
def printCompareDictWCorrections(DFreqDiff, DText, DWiki, DWrongCorrectWord, DWrongCorrectLemma, SFOut):
    fOut = open(SFOut, 'w')
    countEntries = 0
    countCorrLem = 0
    countCorrWord = 0
    for key, val in sorted(DFreqDiff.items(), key=lambda item: item[1], reverse=True):
        countEntries += 1
        try:
            frqText = DText[key] + 1
        except:
            frqText = 1

        try:
            frqWiki = DWiki[key] + 1
        except:
            frqWiki = 1
        
        SWordCorr = ''
        SLemCorr = ''
        SPoSCorr = ''
        try:
            LFieldsKey = key.split('\t')
            if len(LFieldsKey) == 3:
                SWord = LFieldsKey[0]
                SLem = LFieldsKey[2]
                if SWord in DWrongCorrectWord.keys():
                    SWordCorr = DWrongCorrectWord[SWord]
                    countCorrWord +=1

                if SLem in DWrongCorrectLemma.keys():
                    SLemCorr = DWrongCorrectLemma[SLem]
                    countCorrLem +=1
        except:
            pass
        
        fOut.write(f'{key}\t{val}\t{SWordCorr}\t{SPoSCorr}\t{SLemCorr}\t{frqText}\t{frqWiki}\n')
    fOut.flush()
    print(countEntries, countCorrWord, countCorrLem)
    return


In [ ]:
printCompareDictWCorrections(DFreqDiffWikiFiktion, DhyFiktion, DWikiX, DWrongCorrectWord, DWrongCorrectLemma, '/content/KorpusARM1/stage05/hyFiktion.tsv.txt')
printCompareDictWCorrections(DFreqDiffWikiNatur, DhyNatur, DWikiX, DWrongCorrectWord, DWrongCorrectLemma, '/content/KorpusARM1/stage05/hyNatur.tsv.txt')
printCompareDictWCorrections(DFreqDiffWikiRecht, DhyRecht, DWikiX, DWrongCorrectWord, DWrongCorrectLemma, '/content/KorpusARM1/stage05/hyRecht.tsv.txt')
printCompareDictWCorrections(DFreqDiffWikiAll, DhyAll, DWikiX, DWrongCorrectWord, DWrongCorrectLemma, '/content/KorpusARM1/stage05all.tsv.txt')

20969 145 440
21806 94 292
11064 50 117
43675 181 550


## files to annotate:
https://heibox.uni-heidelberg.de/d/8e44baf95ccb4c728a63/


### checking if there is a frequency difference for an entry

In [ ]:
DFreqDiff = {} # dictionary of frequency differences
lenWiki = 2735468
lenText = 83829
c = 0
for key, val in sorted(DText.items(), key=lambda item: item[1], reverse=True):
    c+=1
    valText = val + 1
    relText = valText / lenText
    try:
        valWiki = DWiki[key] + 1
    except:
        valWiki = 1
    relWiki = valWiki / lenWiki

    diffValue = relText / relWiki
    DFreqDiff[key] = diffValue


In [ ]:
fOut = open('Parfum_Armenian-freq-diff.txt', 'w')
for key, val in sorted(DFreqDiff.items(), key=lambda item: item[1], reverse=True):
    try:
        frqText = DText[key] + 1
    except:
        frqText = 1

    try:
        frqWiki = DWiki[key] + 1
    except:
        frqWiki = 1
    fOut.write(f'{key}\t{val}\t{frqText}\t{frqWiki}\n')
fOut.flush()

In [ ]:
cat texts-vert/* >text-vert-all.vert.txt

In [ ]:
!wc text-vert-all.vert.txt

 112723  338169 3062358 text-vert-all.vert.txt


In [ ]:
DText2 = {}
with open("text-vert-all.vert.txt", 'r') as f:
    for line in f:
        line = line.rstrip()
        try:
            DText2[line] +=1
        except:
            DText2[line] = 1

In [ ]:
DFreqDiff2 = {} # dictionary of frequency differences
lenWiki = 2735468
lenText = 112723
c = 0
for key, val in sorted(DText2.items(), key=lambda item: item[1], reverse=True):
    c+=1
    valText = val + 1
    relText = valText / lenText
    try:
        valWiki = DWiki[key] + 1
    except:
        valWiki = 1
    relWiki = valWiki / lenWiki

    diffValue = relText / relWiki
    DFreqDiff2[key] = diffValue


In [ ]:
fOut = open('text-vert-all-freq-diff.txt', 'w')
for key, val in sorted(DFreqDiff2.items(), key=lambda item: item[1], reverse=True):
    try:
        frqText = DText2[key] + 1
    except:
        frqText = 1

    try:
        frqWiki = DWiki[key] + 1
    except:
        frqWiki = 1
    fOut.write(f'{key}\t{val}\t{frqText}\t{frqWiki}\n')
fOut.flush()

## Reading corrected file and discovering rewrite rules
- common prefix; common suffix
- remaining string to rewrite


In [ ]:
# Dowloading the file with corrections
!wget https://heibox.uni-heidelberg.de/f/82b78c77a7bd4eff955d/?dl=1
!mv index.html?dl=1 Parfum_Armenian-freq-diff-all.tsv

In [ ]:
!head -n 40 Parfum_Armenian-freq-diff-all.tsv

In [ ]:
!wc Parfum_Armenian-freq-diff-all.tsv

In [ ]:
!mv Parfum_Armenian-freq-diff-all.tsv Parfum_Armenian-freq-diff-all-v01.tsv

In [ ]:
# Dowloading the file with corrections
!wget https://heibox.uni-heidelberg.de/f/14706c04a4024b2f937d/?dl=1
!mv index.html?dl=1 Parfum_Armenian-freq-diff-all.tsv

In [ ]:
!wc Parfum_Armenian-freq-diff-all.tsv

  325  2864 26793 Parfum_Armenian-freq-diff-all.tsv


In [ ]:
!head -n 40 Parfum_Armenian-freq-diff-all.tsv

In [ ]:
!tail -n 40 Parfum_Armenian-freq-diff-all.tsv

In [ ]:
LTWrongCorrectW, DWrongCorrectW = readCorrections(1, 4, '/content/Parfum_Armenian-freq-diff-all.tsv', SFOut = 'Parfum_Armenian-freq-diff-WrCoWForm.tsv')
LTWrongCorrectL, DWrongCorrectL = readCorrections(3, 6, '/content/Parfum_Armenian-freq-diff-all.tsv', SFOut = 'Parfum_Armenian-freq-diff-WrCoLems.tsv')
LTWrongCorrectWf, DWrongCorrectWf = readCorrectionsFrq(1, 4, 9, '/content/Parfum_Armenian-freq-diff-all.tsv', SFOut = 'Parfum_Armenian-freq-diff-WrCoWFormF.tsv')
LTWrongCorrectLf, DWrongCorrectLf = readCorrectionsFrq(3, 6, 9, '/content/Parfum_Armenian-freq-diff-all.tsv', SFOut = 'Parfum_Armenian-freq-diff-WrCoLemsF.tsv')

ինճ	ինձ	ինչ
առջնից	առջև	առջևից
ինճ	ինչ	ինձ
172
ինճ	ինչ	ես
գիտել	գիտենալ	իմանալ
206
ինճ	ինձ	ինչ
առջնից	առջև	առջևից
ինճ	ինչ	ինձ
172
ինճ	ինչ	ես
գիտել	գիտենալ	իմանալ
206


In [ ]:
print(LTWrongCorrectW)
print(LTWrongCorrectL)
print(LTWrongCorrectWf)
print(LTWrongCorrectLf)

[('[մերճակա]', '[մերձակա]'), ('[առջն]', '[առջև]'), ('[թեթնություն]', '[թեթևություն]'), ('[Եթենա]', '[եթե նա]'), ('[ննա]', '[նա]'), ('[ճեռքերն]', '[ձեռքից]'), ('[ճայն]', '[ձայն]'), ('[ճեռքով]', '[ձեռքով]'), ('[այլնս]', '[այլևս]'), ('[ետնի]', '[ետևի]'), ('[կեղնները]', '[կեղևները]'), ('[ճկան]', '[ձկան]'), ('[ննա]', '[նա]'), ('[բանաձնի]', '[բանաձևի]'), ('[ճեր]', '[ձեր]'), ('[առնտրական]', '[առևտրական]'), ('[արնելյան]', '[արևելյան]'), ('[կուղնորվի]', '[կուղևորվի]'), ('[նս]', '[ևս]'), ('[հետնեց]', '[հետևել]'), ('[նուխիսկ]', '[նույնիսկ]'), ('[երնույթ]', '[երևույթ]'), ('[քրտնքով]', '[քրտինքով]'), ('[արվարճանում]', '[արվարձանում]'), ('[անճամբ]', '[անձամբ]'), ('[ճայնը]', '[ձայնը]'), ('[ճգվում]', '[ձգվում]'), ('[ճիու]', '[ձիու]'), ('[դարճնում]', '[դարձնում]'), ('[ուղնորվում]', '[ուղևորվում ]'), ('[իջնանատան]', '[իջևանատան]'), ('[ճգտում]', '[ձգտում]'), ('[դրսնորում]', '[դրսևորում]'), ('[արվարճանի]', '[արվարձանի]'), ('[ետնում]', '[ետևում]'), ('[ճնավորված]', '[ձևավորված]'), ('[Հետնաբար]', '[հետևաբար]

In [ ]:
# functions to compare two strings
import os
def getCmnPrefix(S1, S2):
    common = os.path.commonprefix([S1, S2])
    return common


def getCmnSuffix(S1,S2):
    S1r = S1[::-1]
    S2r = S2[::-1]
    commonR = getCmnPrefix(S1r, S2r)
    common = commonR[::-1]
    return common

def getDifInfix(S1, S2, LContext = 0, RContext = 0):
    IPLen = len(getCmnPrefix(S1, S2))
    # print('IPLen', IPLen)
    ISLen = len(getCmnSuffix(S1, S2))
    # print('ISLen', ISLen)
    diffInfix1 = S1[IPLen-LContext:-1*(ISLen)+RContext]
    # print('S1', S1, IPLen-LContext, -1*(ISLen)+RContext)
    diffInfix2 = S2[IPLen-LContext:-1*(ISLen)+RContext]
    # print('S2', S2, IPLen-LContext, -1*(ISLen)+RContext)
    return(diffInfix1, diffInfix2)

def getDifSuffix(S1, S2, LContext = 0):
    # used if common suffix == ''
    IPLen = len(getCmnPrefix(S1, S2))
    # print('IPLen', IPLen)
    ISLen = len(getCmnSuffix(S1, S2))
    # print('ISLen', ISLen)
    if ISLen == 0:
        diffSuffix1 = S1[IPLen-LContext:]
        diffSuffix2 = S2[IPLen-LContext:]
    else:
        diffSuffix1 = ''
        diffSuffix2 = ''
    return(diffSuffix1, diffSuffix2)

def getDifPrefix(S1, S2, RContext = 0):
    # used if common suffix == ''
    IPLen = len(getCmnPrefix(S1, S2))
    # print('IPLen', IPLen)
    ISLen = len(getCmnSuffix(S1, S2))
    # print('ISLen', ISLen)
    if IPLen == 0:
        diffPrefix1 = S1[:-1*(ISLen)+RContext]
        diffPrefix2 = S2[:-1*(ISLen)+RContext]
    else:
        diffPrefix1 = ''
        diffPrefix2 = ''
       
    return(diffPrefix1, diffPrefix2)

def getDiff(S1, S2, LContext = 0, RContext = 0):
    IPLen = len(getCmnPrefix(S1, S2))
    ISLen = len(getCmnSuffix(S1, S2))
    if IPLen and ISLen:
        diff1 = S1[IPLen-LContext:-1*(ISLen)+RContext]
        diff2 = S2[IPLen-LContext:-1*(ISLen)+RContext]
    elif IPLen: # there is a common prefix, find different suffixes
        diff1 = S1[IPLen-LContext:]
        diff2 = S2[IPLen-LContext:]
    elif ISLen: # there is a common suffix, find different prefixes
        diff1 = S1[:-1*(ISLen)+RContext]
        diff2 = S2[:-1*(ISLen)+RContext]
    else:
        diff1 = S1
        diff2 = S2
    return (diff1, diff2)



In [ ]:
# first attempt, not used...
'''
def getDifInfix2X(S1, S2, LContext = 0, RContext = 0):
    IPLen = len(getCmnPrefix(S1, S2))
    ISLen = len(getCmnSuffix(S1, S2))

    if LContext and RContext:
        diffInfix1 = S1[IPLen-LContext:-1*(ISLen)+RContext]
        diffInfix2 = S2[IPLen-LContext:-1*(ISLen)+RContext]
        print('both')
    elif LContext:
        diffInfix1 = S1[IPLen-LContext:-1*(ISLen)]
        diffInfix2 = S2[IPLen-LContext:-1*(ISLen)]
        print('left')
    elif RContext:
        diffInfix1 = S1[IPLen:-1*(ISLen+RContext)]
        diffInfix2 = S2[IPLen:-1*(ISLen+RContext)]
        print('right')
    else:
        diffInfix1 = S1[IPLen:-1*ISLen]
        diffInfix2 = S2[IPLen:-1*ISLen]
        print('no-context')

    return(diffInfix1, diffInfix2)



def getDifPrefix(S1, S2, RContext = 0):
    # used if common prefix == ''
    IPLen = len(getCmnPrefix(S1, S2))
    # print('IPLen', IPLen)
    ISLen = len(getCmnSuffix(S1, S2))
    # print('ISLen', ISLen)



'''

In [ ]:
# testing
P12 = getCmnPrefix('перепливи', 'перелови')
S12 = getCmnSuffix('перепливи', 'перелови')
# I1, I2 = getDifInfix('перепливи', 'перелови')
# print(P12, S12)
# print(I1, I2)

I1, I2 = getDifInfix('перепливи', 'перелови', LContext = 0, RContext = 0)
print(I1, I2)

S1, S2 = getDifSuffix('розгубився', 'розгубивсь', LContext = 0)
print(S1, S2)
S1, S2 = getDifSuffix('розгубився', 'розгубивс', LContext = 0)
print(S1, S2)

S1, S2 = getDifPrefix('вловив', 'зловив', RContext = 0)
print(S1, S2)
S1, S2 = getDifPrefix('ловив', 'зловив', RContext = 0)
print(S1, S2)

print('new version....')
I1, I2 = getDiff('перепливи', 'перелови', LContext = 1, RContext = 1)
print(I1, I2)

S1, S2 = getDiff('розгубився', 'розгубивсь', LContext = 1)
print(S1, S2)
S1, S2 = getDiff('розгубився', 'розгубивс', LContext = 1)
print(S1, S2)

S1, S2 = getDiff('вловив', 'зловив', RContext = 1)
print(S1, S2)
S1, S2 = getDiff('ловив', 'зловив', RContext = 1)
print(S1, S2)



пли ло
я ь
я 
в з
 з
new version....
еплив елов
ся сь
ся с
вл зл
л зл


In [ ]:
FOut = open('Parfum_Armenian-freq-diff-Changes2.txt', 'w')

DChanges = {}
for SWrong, SRight in LTWrongCorrectW:
    P12 = getCmnPrefix(SWrong, SRight)
    S12 = getCmnSuffix(SWrong, SRight)
    I1, I2 = getDiff(SWrong, SRight, LContext = 2, RContext =2)
    try:
        DChanges[(I1, I2)] += 1
    except:
        DChanges[(I1, I2)] = 1

    print(SWrong, '(' , P12, '<', I1, '|', I2, '>', S12, ')', SRight)
    FOut.write(f'{SWrong} ( {P12} < {I1} | {I2} > {S12} ) {SRight}\n')

for SWrong, SRight in LTWrongCorrectL:
    P12 = getCmnPrefix(SWrong, SRight)
    S12 = getCmnSuffix(SWrong, SRight)
    I1, I2 = getDiff(SWrong, SRight, LContext = 2, RContext = 2)
    try:
        DChanges[(I1, I2)] += 1
    except:
        DChanges[(I1, I2)] = 1

    print(SWrong, '(' , P12, '<', I1, '|', I2, '>', S12, ')', SRight)
    FOut.write(f'{SWrong} ( {P12} < {I1} | {I2} > {S12} ) {SRight}\n')
FOut.flush()


FOut = open('Parfum_Armenian-freq-diff-ChangeDict2.txt', 'w')
for key, value in sorted(DChanges.items(), key=lambda item: item[1], reverse=True):
    L, R = key
    FOut.write(f'{L}\t{R}\t{value}\n')

FOut.flush()


In [ ]:
## the same with integration of frq information:
FOut = open('ChangesF2.txt', 'w')

DChanges = {}
for SWrong, SRight, SFrq in LTWrongCorrectWf:
    try:
        IFrq = int(SFrq)
    except:
        IFrq = 1
    P12 = getCmnPrefix(SWrong, SRight)
    S12 = getCmnSuffix(SWrong, SRight)
    I1, I2 = getDiff(SWrong, SRight, LContext = 2, RContext =2)
    try:
        DChanges[(I1, I2)] += IFrq
    except:
        DChanges[(I1, I2)] = IFrq

    print(SWrong, '(' , P12, '<', I1, '|', I2, '>', S12, ')', SRight)
    FOut.write(f'{SWrong} ( {P12} < {I1} | {I2} > {S12} ) {SRight}\n')

for SWrong, SRight, SFrq in LTWrongCorrectLf:
    try:
        IFrq = int(SFrq)
    except:
        IFrq = 1
    P12 = getCmnPrefix(SWrong, SRight)
    S12 = getCmnSuffix(SWrong, SRight)
    I1, I2 = getDiff(SWrong, SRight, LContext = 2, RContext = 2)
    try:
        DChanges[(I1, I2)] += IFrq
    except:
        DChanges[(I1, I2)] = IFrq

    print(SWrong, '(' , P12, '<', I1, '|', I2, '>', S12, ')', SRight)
    FOut.write(f'{SWrong} ( {P12} < {I1} | {I2} > {S12} ) {SRight}\n')
FOut.flush()


FOut = open('ChangeDictF2.txt', 'w')
for key, value in sorted(DChanges.items(), key=lambda item: item[1], reverse=True):
    L, R = key
    if L != '' and R != '' and L != R:
        FOut.write(f'{L}\t{R}\t{value}\n')

FOut.flush()

In [ ]:
# !cat /content/ChangeDictF1.txt | tail -n+2 >/content/ChangeDictF1a.txt
# !cat /content/ChangeDictF2.txt | tail -n+2 >/content/ChangeDictF2a.txt


In [ ]:
# !mv /content/ChangeDictF1.txt /content/KorpusARM1/
# !mv /content/ChangeDictF2.txt /content/KorpusARM1/


In [ ]:
!cat ChangeDict* >ChangesDFAll.txt

In [ ]:
# all substrings of a string
# https://www.geeksforgeeks.org/python-get-all-substrings-of-given-string/

'''
# Python3 code to demonstrate working of
# Get all substrings of string
# Using list comprehension + string slicing
 
# initializing string
test_str = "Geeks"
 
# printing original string
print("The original string is : " + str(test_str))
 
# Get all substrings of string
# Using list comprehension + string slicing
res = [test_str[i: j] for i in range(len(test_str))
          for j in range(i + 1, len(test_str) + 1)]
 
# printing result
print("All substrings of string are : " + str(res))

'''

In [ ]:
# now we read wikipedia file into a frq dictionary, and generate all substrings for each string, and create a frq dictionary of substrings

def corp2tokLim2frqDict(SFIn):
    DFrq = {}
    with open(SFIn) as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip()
            # LLine = SLine.split('\t')
            LLine = re.split('[ ,:\.;\-\(\)\?\!\[\]]+', SLine)
            for tok in LLine:
                tokLim = f'[{tok}]'
                try:
                    DFrq[tokLim]+=1
                except:
                    DFrq[tokLim]=1
    return DFrq

def printFrqDict(DFrq, SFOut):
    FOut = open(SFOut, 'w')
    count = 0
    for key, val in sorted(DFrq.items(), key=lambda item: item[1], reverse=True):
        count+=1
        FOut.write(f'{count}\t{key}\t{val}\n')
    FOut.flush()

In [ ]:
# runs 1 min
DTokLimFrqWikipedia = corp2tokLim2frqDict('/content/hywiki-20221101-pages-articles.txt')
printFrqDict(DTokLimFrqWikipedia, 'hywiki-20221101-frq-dict.txt')

In [ ]:
!wc hywiki-20221101-frq-dict.txt

 2586453  7759467 78654850 hywiki-20221101-frq-dict.txt


In [ ]:
!head -n 40 hywiki-20221101-frq-dict.txt

In [ ]:
def readChangesLeft2dict(SFIn):
    DFrq = {}
    DRepl = {}
    with open(SFIn) as FIn:
        for SLine in FIn:
            SLine = SLine.rstrip()
            LLine = SLine.split('\t')
            try:
                L = LLine[0]
                R = LLine[1]
                SFrq = LLine[2]
                IFrq = int(SFrq)
                DFrq[L]=IFrq
                DRepl[L]= R

            except:
                pass

    return DFrq, DRepl


In [ ]:
DFrqChangesL, DFrqChangesReplacements = readChangesLeft2dict('/content/ChangesDFAll.txt')
printFrqDict(DFrqChangesL, 'ChangesDFLeftAll.txt')

In [ ]:
# initializing string
test_str = "Geeks"
 
# printing original string
print("The original string is : " + str(test_str))
 
# Get all substrings of string
# Using list comprehension + string slicing
res = [test_str[i: j] for i in range(len(test_str))
          for j in range(i + 1, len(test_str) + 1)]
 
# printing result
print("All substrings of string are : " + str(res))

The original string is : Geeks
All substrings of string are : ['G', 'Ge', 'Gee', 'Geek', 'Geeks', 'e', 'ee', 'eek', 'eeks', 'e', 'ek', 'eks', 'k', 'ks', 's']


In [ ]:
def frqDict2substrDict(DFrq,FrqThreshold=0):
    DFrqSubStr = {}
    for test_str, frq in DFrq.items():
        if frq <= FrqThreshold: continue
        LSubStr = [test_str[i: j] for i in range(len(test_str)) for j in range(i + 1, len(test_str) + 1)]
        for el in LSubStr:
            try:
                DFrqSubStr[el] += frq
            except:
                DFrqSubStr[el] = frq
    return DFrqSubStr

In [ ]:
# runs approx 1.5 min
DFrqSubstringsWiki = frqDict2substrDict(DTokLimFrqWikipedia, FrqThreshold = 1)
printFrqDict(DFrqSubstringsWiki, '/content/hywiki-20221101-frq-dict-substrings.txt')

In [ ]:
!wc /content/hywiki-20221101-frq-dict-substrings.txt

 12755762  38267464 361186819 /content/hywiki-20221101-frq-dict-substrings.txt


In [ ]:
!head -n 40 /content/hywiki-20221101-frq-dict-substrings.txt

In [ ]:
FOutLeft = open('ChangesDLeftNinWiki.txt', 'w')
for L, frqL in sorted(DFrqChangesL.items(), key=lambda item: item[1], reverse=True):
    if L not in DFrqSubstringsWiki.keys():
        try:
            R = DFrqChangesReplacements[L]
        except:
            R = 'NONE'
        FOutLeft.write(f'{L}\t{R}\t{str(frqL)}\n')

FOutLeft.flush()